In [1]:
import tensorflow as tf
import numpy as np
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import heapq
import warnings as wg
wg.filterwarnings("ignore")

from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

In [2]:
file = open('1661-0.txt', encoding = 'utf8').read().lower()

In [3]:
print('Corpus lenth:', len(file))

Corpus lenth: 581888


In [4]:
file = open('1661-0.txt', 'r', encoding = 'utf8')
lines = []

for i in file:
  lines.append(i)


In [5]:
data= ""

for i in lines:
  data = ''.join(lines)

data = data.replace('\n','').replace('\r','').replace('\ufeff','')
data[:360]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan DoyleThis eBook is for the use of anyone anywhere at no cost and withalmost no restrictions whatsoever.  You may copy it, give it away orre-use it under the terms of the Project Gutenberg License includedwith this eBook or online at www.gutenberg.netTitle: The Adventures of Sherlock Holme"

In [6]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
new_data = data.translate(translator)

new_data[:500]

'Project Gutenberg s The Adventures of Sherlock Holmes  by Arthur Conan DoyleThis eBook is for the use of anyone anywhere at no cost and withalmost no restrictions whatsoever   You may copy it  give it away orre use it under the terms of the Project Gutenberg License includedwith this eBook or online at www gutenberg netTitle  The Adventures of Sherlock HolmesAuthor  Arthur Conan DoyleRelease Date  November 29  2002  EBook  1661 Last Updated  May 20  2019Language  EnglishCharacter set encoding  U'

In [7]:
z = []

for i in data.split():
  if i not in z:
    z.append(i)

data = ' '.join(z)
data[:500]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan DoyleThis eBook is for the use anyone anywhere at no cost and withalmost restrictions whatsoever. You may copy it, give it away orre-use under terms Gutenberg License includedwith this or online www.gutenberg.netTitle: HolmesAuthor: DoyleRelease Date: November 29, 2002 [EBook #1661]Last Updated: May 20, 2019Language: EnglishCharacter set encoding: UTF-8*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES SHERLOCK HOLMES *"

In [8]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data])

#
pickle.dump(tokenizer, open('tokenizer1.pk1', 'wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[734, 3285, 20, 516, 105, 134, 10, 175, 735, 3286]

In [9]:
vocab_size = len(tokenizer.word_index) + 1

In [10]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  24990


array([[ 734, 3285],
       [3285,   20],
       [  20,  516],
       [ 516,  105],
       [ 105,  134],
       [ 134,   10],
       [  10,  175],
       [ 175,  735],
       [ 735, 3286],
       [3286, 3287]])

In [11]:
X= []
y= []

for i in sequences:
  X.append(i[0])
  y.append(i[1])

X = np.array(X)
y = np.array(y)

print("the data is:", X[:5])
print("the responses are:",y[:5])
      

the data is: [ 734 3285   20  516  105]
the responses are: [3285   20  516  105  134]


In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
model =  Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences= True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation= 'softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             136000    
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 13600)             13613600  
                                                                 
Total params: 26,798,600
Trainable params: 26,798,600
Non-trainable params: 0
_________________________________________________________________


In [19]:
checkpoint = ModelCheckpoint("nextworld1.h5", monitor='loss', verbose=1,
                             save_best_only= True, mode='auto' )
reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)

logdir = 'logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

In [17]:
model.fit(X, y, epochs=40, batch_size=150, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/40
167/167 [==============================] - ETA: 0s - loss: 9.3991
Epoch 1: loss improved from inf to 9.39912, saving model to nextworld1.h5
167/167 [==============================] - 15s 45ms/step - loss: 9.3991 - lr: 0.0010
Epoch 2/40
167/167 [==============================] - ETA: 0s - loss: 9.0008
Epoch 2: loss improved from 9.39912 to 9.00083, saving model to nextworld1.h5
167/167 [==============================] - 7s 44ms/step - loss: 9.0008 - lr: 0.0010
Epoch 3/40
166/167 [============================>.] - ETA: 0s - loss: 8.7687
Epoch 3: loss improved from 9.00083 to 8.76936, saving model to nextworld1.h5
167/167 [==============================] - 8s 45ms/step - loss: 8.7694 - lr: 0.0010
Epoch 4/40
167/167 [==============================] - ETA: 0s - loss: 8.5961
Epoch 4: loss improved from 8.76936 to 8.59610, saving model to nextworld1.h5
167/167 [==============================] - 7s 45ms/step - loss: 8.5961 - lr: 0.0010
Epoch 5/40
167/167 [===========================

In [21]:
model = load_model('nextworld1.h5')
tokenizer = pickle.load(open('tokenizer1.pk1','rb'))

def predict_next_word(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predcited_word = ""

  for key,value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predcited_word


In [27]:
while(True):
  text= input('enter your line: ')

  if text=='0':
    print('execution completed......')
    break

  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)

      predict_next_word(model, tokenizer, text)

    except Exception as e:
      print('error occured',e)
      continue



enter your line: 0
execution completed......
